# Aspect-Based Sentiment Analysis (ABSA) Pipeline
Ce notebook implémente un pipeline ABSA complet qui :
1. Identifie les aspects dans le texte
2. Analyse le sentiment pour chaque aspect identifié
3. Combine les résultats pour une analyse complète

In [ ]:
!pip install --upgrade datasets numpy
!pip install --upgrade transformers
!pip install --upgrade torch
!pip install --upgrade scikit-learn
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 62.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade datasets numpy
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 112.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [ ]:
from transformers import AutoTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import pipeline
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np
import torch
import spacy
import re
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Charger le modèle spaCy pour l'extraction d'aspects
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Chargement des données
df = pd.read_csv("absa_balanced_augmented_dataset_en (1).csv")
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nLabel distribution:")
print(df['label'].value_counts())
print("\nFirst few rows:")
print(df.head())

Dataset shape: (17970, 6)
Columns: ['review_id', 'sentence', 'aspect', 'original_rating', 'sentiment_initial', 'label']

Label distribution:
label
negative    5990
neutral     5990
positive    5990
Name: count, dtype: int64

First few rows:
      review_id                                           sentence     aspect  \
0          2077  Poor fabric quality Looks great in the picture...       mail   
1  gen_neu_3928            My opinion on the reference is neutral.  reference   
2         15794  So disappointing I love the photo of this and ...    reviews   
3         14696                              the fabric is pretty.     pretty   
4   gen_neg_629                          The large is problematic.      large   

   original_rating sentiment_initial     label  
0                1          negative  negative  
1                2          negative   neutral  
2                1          negative  positive  
3                3           neutral  positive  
4                5         

In [ ]:
class AspectExtractor:
    def __init__(self):
        # Dictionnaire d'aspects prédéfinis pour différents domaines
        self.aspect_keywords = {
            'fit': ['fit', 'size', 'sizing', 'tight', 'loose', 'snug', 'true to size'],
            'material': ['fabric', 'material', 'texture', 'quality', 'soft', 'rough', 'sheer', 'durable'],
            'style': ['design', 'look', 'style', 'fashion', 'cut', 'pattern', 'color', 'trendy'],
            'price': ['price', 'cost', 'value', 'expensive', 'cheap', 'worth', 'overpriced', 'affordable'],
            'delivery': ['delivery', 'shipping', 'arrival', 'delay', 'on time', 'fast', 'late'],
            'customer_service': ['support', 'customer service', 'help', 'return', 'refund', 'exchange', 'response'],
            'general': ['experience', 'purchase', 'recommend', 'satisfaction', 'overall', 'product']
        }

    def extract_aspects_from_text(self, text):
        """Extrait les aspects d'un texte donné"""
        text_lower = text.lower()
        found_aspects = []

        # Recherche par mots-clés
        for aspect_category, keywords in self.aspect_keywords.items():
            for keyword in keywords:
                if keyword in text_lower:
                    found_aspects.append(aspect_category)
                    break

        # Extraction d'entités avec spaCy
        doc = nlp(text)
        entities = [ent.text.lower() for ent in doc.ents if ent.label_ in ['PRODUCT', 'ORG', 'PERSON']]

        # Si aucun aspect trouvé, utiliser 'general'
        if not found_aspects:
            found_aspects = ['general']

        return list(set(found_aspects))  # Retourner aspects uniques

    def get_aspect_from_dataset(self, row):
        """Utilise l'aspect du dataset s'il existe, sinon extrait du texte"""
        if 'aspect' in row and pd.notna(row['aspect']) and row['aspect'].strip():
            return row['aspect'].lower()
        else:
            aspects = self.extract_aspects_from_text(row['sentence'])
            return aspects[0] if aspects else 'general'

# Test de l'extracteur d'aspects
extractor = AspectExtractor()
test_text = "The dress is gorgeous, but the customer service was terrible and the price was way too high."
print(f"Text: {test_text}")
print(f"Extracted aspects: {extractor.extract_aspects_from_text(test_text)}")

Text: The dress is gorgeous, but the customer service was terrible and the price was way too high.
Extracted aspects: ['price', 'customer_service']


In [ ]:
# Préparation des données pour l'entraînement ABSA
def prepare_absa_data(df):
    """Prépare les données pour l'entraînement ABSA"""
    extractor = AspectExtractor()

    # Créer une copie du dataframe
    df_absa = df.copy()

    # Extraire ou utiliser les aspects existants
    df_absa['extracted_aspect'] = df_absa.apply(extractor.get_aspect_from_dataset, axis=1)

    # Mapping des labels de sentiment
    label_to_id = {'positive': 0, 'negative': 1, 'neutral': 2}
    id_to_label = {v: k for k, v in label_to_id.items()}
    df_absa["label_id"] = df_absa["label"].map(label_to_id)

    return df_absa, label_to_id, id_to_label

df_absa, label_to_id, id_to_label = prepare_absa_data(df)
print(f"Prepared ABSA dataset shape: {df_absa.shape}")
print("\nAspect distribution:")
print(df_absa['extracted_aspect'].value_counts())

Prepared ABSA dataset shape: (17970, 8)

Aspect distribution:
extracted_aspect
product       664
small         277
bit           232
large         193
retailer      193
             ... 
sooooooooo      1
exquisite       1
gown            1
swimwear        1
armpits         1
Name: count, Length: 1999, dtype: int64


In [ ]:
# Division des données
train_val_df, test_df = train_test_split(df_absa, test_size=0.10, stratify=df_absa['label_id'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.111, stratify=train_val_df['label_id'], random_state=42)

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

# Créer les datasets HuggingFace
hf_dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True))
})

print("\nDataset splits created successfully!")

Train set size: 14377
Validation set size: 1796
Test set size: 1797

Dataset splits created successfully!


In [ ]:
train_df.head()


,review_id,sentence,aspect,original_rating,sentiment_initial,label,extracted_aspect,label_id
6512,102,just received and it fits nicely-a little on t...,big,5,positive,negative,big,1
6103,14512,the only downside is that the bottom is not he...,only,5,positive,negative,only,1
14717,gen_neg_2251,Retailer was truly disappointing.,retailer,4,positive,negative,retailer,1
12812,14122,"Great, comfortable jacket Love this jacket.",comfortable,5,positive,positive,comfortable,0
8513,5867,so sad:(,sad,2,negative,negative,sad,1


In [ ]:
# Tokenisation pour ABSA (sentence + aspect)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function_absa(examples):
    """Tokenise les phrases avec leurs aspects pour ABSA"""
    # Utiliser l'aspect extrait si disponible, sinon utiliser l'aspect original
    aspects = []
    for i in range(len(examples["sentence"])):
        if 'extracted_aspect' in examples and examples['extracted_aspect'][i]:
            aspects.append(examples['extracted_aspect'][i])
        elif 'aspect' in examples and examples['aspect'][i]:
            aspects.append(examples['aspect'][i])
        else:
            aspects.append('general')

    # Tokeniser sentence + aspect
    return tokenizer(
        examples["sentence"],
        aspects,
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Appliquer la tokenisation
tokenized_datasets = hf_dataset.map(tokenize_function_absa, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label_id", "labels")

# Supprimer les colonnes non nécessaires
columns_to_remove = [col for col in tokenized_datasets["train"].column_names
                    if col not in ["input_ids", "attention_mask", "labels"]]
tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)
tokenized_datasets.set_format("torch")

print("Tokenization completed!")
print(f"Features: {tokenized_datasets['train'].features}")

Map:   0%|          | 0/14377 [00:00<?, ? examples/s]

Map:   0%|          | 0/1796 [00:00<?, ? examples/s]

Map:   0%|          | 0/1797 [00:00<?, ? examples/s]

Tokenization completed!
Features: {'labels': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [ ]:
# Configuration du modèle ABSA
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3,
    id2label=id_to_label,
    label2id=label_to_id
)

def compute_metrics(pred):
    """Calcule les métriques d'évaluation"""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

print("Model configured for ABSA!")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model configured for ABSA!


In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results_absa_pipeline",
    eval_strategy="epoch",
    save_strategy="epoch",
    do_eval=True,
    learning_rate=2e-5,  # Learning rate légèrement plus élevé pour ABSA
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs_absa",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    warmup_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print("Training setup completed!")

Training setup completed!


In [ ]:
# Entraînement du modèle
print("Starting ABSA model training...")
trainer.train()
print("Training completed!")

# Évaluation finale
print("\n=" * 50)
print("FINAL EVALUATION ON TEST SET")
print("=" * 50)
test_results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:")
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")

# Sauvegarde du modèle
model_path = "./best_distilbert_absa_pipeline_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"\nModel and tokenizer saved to: {model_path}")

Starting ABSA model training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.388100,0.368904,0.853007,0.852688,0.862792,0.853007
2,0.295900,0.381994,0.869710,0.869716,0.869815,0.869710
3,0.224400,0.454807,0.860802,0.861023,0.861926,0.860802
4,0.168400,0.521165,0.857461,0.857498,0.857697,0.857461


Training completed!

=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
FINAL EVALUATION ON TEST SET


Test Results:
eval_loss: 0.4151
eval_accuracy: 0.8609
eval_f1: 0.8611
eval_precision: 0.8624
eval_recall: 0.8609
eval_runtime: 1.5958
eval_samples_per_second: 1126.0490
eval_steps_per_second: 35.7180
epoch: 4.0000

Model and tokenizer saved to: ./best_distilbert_absa_pipeline_model


In [ ]:
# Pipeline ABSA complet
class ABSAPipeline:
    def __init__(self, model_path, tokenizer_path=None):
        if tokenizer_path is None:
            tokenizer_path = model_path

        self.model = DistilBertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        self.aspect_extractor = AspectExtractor()
        self.model.eval()

    def predict_sentiment_for_aspect(self, sentence, aspect):
        """Prédit le sentiment pour un aspect spécifique"""
        encoding = self.tokenizer(
            sentence,
            aspect,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=128
        )

        with torch.no_grad():
            outputs = self.model(**encoding)

        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        pred_id = int(torch.argmax(logits, dim=1).item())
        confidence = float(probabilities[0][pred_id].item())

        return {
            "aspect": aspect,
            "sentiment": self.model.config.id2label[pred_id],
            "confidence": confidence,
            "probabilities": {
                "positive": float(probabilities[0][0].item()),
                "negative": float(probabilities[0][1].item()),
                "neutral": float(probabilities[0][2].item())
            }
        }

    def analyze_text(self, text, aspects=None):
        """Analyse complète ABSA d'un texte"""
        if aspects is None:
            # Extraire automatiquement les aspects
            aspects = self.aspect_extractor.extract_aspects_from_text(text)

        if isinstance(aspects, str):
            aspects = [aspects]

        results = []
        for aspect in aspects:
            result = self.predict_sentiment_for_aspect(text, aspect)
            results.append(result)

        return {
            "text": text,
            "absa_results": results,
            "summary": self._create_summary(results)
        }

    def _create_summary(self, results):
        """Crée un résumé des résultats ABSA"""
        sentiment_counts = defaultdict(int)
        total_confidence = 0

        for result in results:
            sentiment_counts[result['sentiment']] += 1
            total_confidence += result['confidence']

        avg_confidence = total_confidence / len(results) if results else 0
        dominant_sentiment = max(sentiment_counts.items(), key=lambda x: x[1])[0] if sentiment_counts else 'neutral'

        return {
            "dominant_sentiment": dominant_sentiment,
            "average_confidence": avg_confidence,
            "sentiment_distribution": dict(sentiment_counts),
            "total_aspects": len(results)
        }

# Initialiser le pipeline
absa_pipeline = ABSAPipeline(model_path)
print("ABSA Pipeline initialized successfully!")

ABSA Pipeline initialized successfully!


In [ ]:
# Tests du pipeline ABSA
print("=" * 60)
print("TESTING ABSA PIPELINE")
print("=" * 60)

test_examples = [
    "The dress fabric is very soft, but the sizing is completely off and the delivery was delayed.",
    "Beautiful blouse and the colors are exactly as shown, I love it!",
    "The top looks nice but the stitching quality is poor.",
    "Overall, the pants are fine. Not amazing, but not disappointing either.",
    "The heels are stunning and comfortable, but they are way too expensive for the quality."
]

for i, text in enumerate(test_examples, 1):
    print(f"\n--- Example {i} ---")
    print(f"Text: {text}")

    # Analyse automatique
    result = absa_pipeline.analyze_text(text)

    print("\nABSA Results:")
    for absa_result in result['absa_results']:
        print(f"  Aspect: {absa_result['aspect']}")
        print(f"  Sentiment: {absa_result['sentiment']} (confidence: {absa_result['confidence']:.3f})")
        print()

    print(f"Summary:")
    summary = result['summary']
    print(f"  Dominant sentiment: {summary['dominant_sentiment']}")
    print(f"  Average confidence: {summary['average_confidence']:.3f}")
    print(f"  Total aspects analyzed: {summary['total_aspects']}")
    print("-" * 40)

TESTING ABSA PIPELINE

--- Example 1 ---
Text: The dress fabric is very soft, but the sizing is completely off and the delivery was delayed.

ABSA Results:
  Aspect: material
  Sentiment: negative (confidence: 0.970)

  Aspect: delivery
  Sentiment: negative (confidence: 0.987)

  Aspect: fit
  Sentiment: negative (confidence: 0.914)

Summary:
  Dominant sentiment: negative
  Average confidence: 0.957
  Total aspects analyzed: 3
----------------------------------------

--- Example 2 ---
Text: Beautiful blouse and the colors are exactly as shown, I love it!

ABSA Results:
  Aspect: style
  Sentiment: positive (confidence: 0.984)

Summary:
  Dominant sentiment: positive
  Average confidence: 0.984
  Total aspects analyzed: 1
----------------------------------------

--- Example 3 ---
Text: The top looks nice but the stitching quality is poor.

ABSA Results:
  Aspect: material
  Sentiment: negative (confidence: 0.987)

  Aspect: style
  Sentiment: negative (confidence: 0.975)

Summary:
 

In [ ]:
# Test avec des aspects spécifiques
print("\n" + "=" * 60)
print("TESTING WITH CUSTOM ASPECTS")
print("=" * 60)

custom_text = "The jacket material feels premium and the cut is elegant, but the shipping took forever."
custom_aspects = ["material", "style", "delivery"]

print(f"Text: {custom_text}")
print(f"Custom aspects: {custom_aspects}")

result = absa_pipeline.analyze_text(custom_text, custom_aspects)

print("\nResults:")
for absa_result in result['absa_results']:
    print(f"Aspect: {absa_result['aspect']} -> Sentiment: {absa_result['sentiment']} ({absa_result['confidence']:.3f})")
    probs = absa_result['probabilities']
    print(f"  Probabilities: Pos:{probs['positive']:.3f}, Neg:{probs['negative']:.3f}, Neu:{probs['neutral']:.3f}")
    print()


TESTING WITH CUSTOM ASPECTS
Text: The jacket material feels premium and the cut is elegant, but the shipping took forever.
Custom aspects: ['material', 'style', 'delivery']

Results:
Aspect: material -> Sentiment: positive (0.921)
  Probabilities: Pos:0.921, Neg:0.028, Neu:0.051

Aspect: style -> Sentiment: positive (0.930)
  Probabilities: Pos:0.930, Neg:0.021, Neu:0.049

Aspect: delivery -> Sentiment: positive (0.883)
  Probabilities: Pos:0.883, Neg:0.044, Neu:0.073



In [ ]:
# Évaluation détaillée sur le test set
print("\n" + "=" * 60)
print("DETAILED EVALUATION ON TEST SET")
print("=" * 60)

# Prédictions sur le test set
test_predictions = trainer.predict(tokenized_datasets["test"])
y_pred = test_predictions.predictions.argmax(-1)
y_true = test_predictions.label_ids

# Rapport de classification détaillé
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['positive', 'negative', 'neutral']))

# Matrice de confusion
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print("     Pos  Neg  Neu")
for i, row in enumerate(cm):
    print(f"{['Pos', 'Neg', 'Neu'][i]}: {row}")


DETAILED EVALUATION ON TEST SET


Classification Report:
              precision    recall  f1-score   support

    positive       0.86      0.84      0.85       599
    negative       0.90      0.86      0.88       599
     neutral       0.82      0.88      0.85       599

    accuracy                           0.86      1797
   macro avg       0.86      0.86      0.86      1797
weighted avg       0.86      0.86      0.86      1797


Confusion Matrix:
     Pos  Neg  Neu
Pos: [504  32  63]
Neg: [ 36 513  50]
Neu: [ 44  25 530]


In [ ]:
# Sauvegarde finale et instructions d'utilisation
import pickle

# Sauvegarder l'extracteur d'aspects
with open('./aspect_extractor.pkl', 'wb') as f:
    pickle.dump(absa_pipeline.aspect_extractor, f)

print("=" * 60)
print("ABSA PIPELINE TRAINING COMPLETED!")
print("=" * 60)
print("\nFiles saved:")
print(f"- Model: {model_path}")
print(f"- Aspect extractor: ./aspect_extractor.pkl")
print("\nTo use the pipeline in another notebook:")
print("""
# Charger le pipeline
absa_pipeline = ABSAPipeline('./best_distilbert_absa_pipeline_model')

# Analyser un texte
result = absa_pipeline.analyze_text("Your text here")
print(result)

# Ou avec des aspects spécifiques
result = absa_pipeline.analyze_text("Your text here", ["food", "service", "price"])
""")

print("\nPipeline ready for production use!")

ABSA PIPELINE TRAINING COMPLETED!

Files saved:
- Model: ./best_distilbert_absa_pipeline_model
- Aspect extractor: ./aspect_extractor.pkl

To use the pipeline in another notebook:

# Charger le pipeline
absa_pipeline = ABSAPipeline('./best_distilbert_absa_pipeline_model')

# Analyser un texte
result = absa_pipeline.analyze_text("Your text here")
print(result)

# Ou avec des aspects spécifiques
result = absa_pipeline.analyze_text("Your text here", ["food", "service", "price"])


Pipeline ready for production use!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split

# 1. Préparation des données
X = df_absa["sentence"]               # Textes des avis
y = df_absa["label"]                  # Labels (positive, negative, neutral)

# 2. Split train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. Vectorisation TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 4. Définir les modèles à tester
models = {
    "SVM": LinearSVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

# 5. Entraînement et évaluation
results = {}

for name, model in models.items():
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)

    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred))

    f1 = f1_score(y_test, y_pred, average='weighted')
    results[name] = f1

# 6. Résumé comparatif
print("\n=== Résumé des F1-scores pondérés ===")
for name, score in results.items():
    print(f"{name}: {score:.4f}")



=== SVM ===
              precision    recall  f1-score   support

    negative       0.81      0.79      0.80      1198
     neutral       0.86      0.78      0.82      1198
    positive       0.73      0.82      0.77      1198

    accuracy                           0.80      3594
   macro avg       0.80      0.80      0.80      3594
weighted avg       0.80      0.80      0.80      3594


=== Random Forest ===
              precision    recall  f1-score   support

    negative       0.87      0.71      0.78      1198
     neutral       0.94      0.73      0.82      1198
    positive       0.65      0.92      0.76      1198

    accuracy                           0.79      3594
   macro avg       0.82      0.79      0.79      3594
weighted avg       0.82      0.79      0.79      3594


=== Logistic Regression ===
              precision    recall  f1-score   support

    negative       0.82      0.80      0.81      1198
     neutral       0.90      0.76      0.82      1198
    positi